# Mini Project: Transfer Learning with Keras

Transfer learning is a machine learning technique where a model trained on one task is used as a starting point to solve a different but related task. Instead of training a model from scratch, transfer learning leverages the knowledge learned from the source task and applies it to the target task. This approach is especially useful when the target task has limited data or computational resources.

In transfer learning, the pre-trained model, also known as the "base model" or "source model," is typically trained on a large dataset and a more general problem (e.g., image classification on ImageNet, a vast dataset with millions of labeled images). The knowledge learned by the base model in the form of feature representations and weights captures common patterns and features in the data.

To perform transfer learning, the following steps are commonly followed:

1. Pre-training: The base model is trained on a source task using a large dataset, which can take a considerable amount of time and computational resources.

2. Feature Extraction: After pre-training, the base model is used as a feature extractor. The last few layers (classifier layers) of the model are discarded, and the remaining layers (feature extraction layers) are retained. These layers serve as feature extractors, producing meaningful representations of the data.

3. Fine-tuning: The feature extraction layers and sometimes some of the earlier layers are connected to a new set of layers, often called the "classifier layers" or "task-specific layers." These layers are randomly initialized, and the model is trained on the target task with a smaller dataset. The weights of the base model can be frozen during fine-tuning, or they can be allowed to be updated with a lower learning rate to fine-tune the model for the target task.

Transfer learning has several benefits:

1. Reduced training time and resource requirements: Since the base model has already learned generic features, transfer learning can save time and resources compared to training a model from scratch.

2. Improved generalization: Transfer learning helps the model generalize better to the target task, especially when the target dataset is small and dissimilar from the source dataset.

3. Better performance: By starting from a model that is already trained on a large dataset, transfer learning can lead to better performance on the target task, especially in scenarios with limited data.

4. Effective feature extraction: The feature extraction layers of the pre-trained model can serve as powerful feature extractors for different tasks, even when the task domains differ.

Transfer learning is commonly used in various domains, including computer vision, natural language processing (NLP), and speech recognition, where pre-trained models are fine-tuned for specific applications like object detection, sentiment analysis, or speech-to-text.

In this mini-project you will perform fine-tuning using Keras with a pre-trained VGG16 model on the CIFAR-10 dataset.

First, import all the libraries you'll need.

In [88]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

The CIFAR-10 dataset is a widely used benchmark dataset in the field of computer vision and machine learning. It stands for the "Canadian Institute for Advanced Research 10" dataset. CIFAR-10 was created by researchers at the CIFAR institute and was originally introduced as part of the Neural Information Processing Systems (NIPS) 2009 competition.

The dataset consists of 60,000 color images, each of size 32x32 pixels, belonging to ten different classes. Each class contains 6,000 images. The ten classes in CIFAR-10 are:

1. Airplane
2. Automobile
3. Bird
4. Cat
5. Deer
6. Dog
7. Frog
8. Horse
9. Ship
10. Truck

The images are evenly distributed across the classes, making CIFAR-10 a balanced dataset. The dataset is divided into two sets: a training set and a test set. The training set contains 50,000 images, while the test set contains the remaining 10,000 images.

CIFAR-10 is often used for tasks such as image classification, object recognition, and transfer learning experiments. The relatively small size of the images and the variety of classes make it a challenging dataset for training machine learning models, especially deep neural networks. It also serves as a good dataset for teaching and learning purposes due to its manageable size and straightforward class labels.

Here are your tasks:

1. Load the CIFAR-10 dataset after referencing the documentation [here](https://keras.io/api/datasets/cifar10/).
2. Normalize the pixel values so they're all in the range [0, 1].
3. Apply One Hot Encoding to the train and test labels using the [to_categorical](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical) function.
4. Further split the the training data into training and validation sets using [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). Use only 10% of the data for validation.  

In [89]:
# Load the CIFAR-10 dataset
CIFAR_10_data = cifar10.load_data()
CIFAR_class_dict = {0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}

In [90]:
# Explore the Data
print(type(CIFAR_10_data))
print(len(CIFAR_10_data))


<class 'tuple'>
2


In [91]:
print(type(CIFAR_10_data[0]))
print(len(CIFAR_10_data[0]))


<class 'tuple'>
2


In [92]:
print(type(CIFAR_10_data[1]))
print(len(CIFAR_10_data[1]))

<class 'tuple'>
2


In [93]:
# The data looks like this at the top level ((0, 1), (0,1))

In [94]:
print(type(CIFAR_10_data[0][0]))
print(len(CIFAR_10_data[0][0]))
print(type(CIFAR_10_data[0][1]))
print(len(CIFAR_10_data[0][1]))

<class 'numpy.ndarray'>
50000
<class 'numpy.ndarray'>
50000


In [95]:
# Data looks like (([0..49999], [0..49999]), (0, 1))
print(type(CIFAR_10_data[1][0]))
print(len(CIFAR_10_data[1][0]))
print(type(CIFAR_10_data[1][1]))
print(len(CIFAR_10_data[1][1]))

<class 'numpy.ndarray'>
10000
<class 'numpy.ndarray'>
10000


In [96]:
# OK I get it now.
# CIFAR_10_data[0][0] is the training data at 50000 values of image data
# CIFAR_10_dat[0][1] is the training data labels at 50000 values
# the next tuple in the top tuple is the test data at 10000 images and 10000 labels

train_images = CIFAR_10_data[0][0]
train_labels = CIFAR_10_data[0][1]
test_images = CIFAR_10_data[1][0]
test_labels = CIFAR_10_data[1][1]

In [97]:
print(type(train_images))
print(len(train_images))
print(type(train_labels))
print(len(train_labels))
print(type(test_images))
print(len(test_images))
print(type(test_labels))
print(len(test_labels))

<class 'numpy.ndarray'>
50000
<class 'numpy.ndarray'>
50000
<class 'numpy.ndarray'>
10000
<class 'numpy.ndarray'>
10000


In [98]:
# normalize the image data from 0 to 1
# assumes 0-255 as the RGB space
train_images = train_images / 255.0
test_images = test_images / 255.0






In [99]:
# test assumption
print(train_images.max())
print(train_images.min())
print(test_images.max())
print(test_images.min())

1.0
0.0
1.0
0.0


In [113]:
# One-hot encode the classification data
print(type(train_labels[0]))
print(type(train_labels[0][0]))
print(train_labels.shape)
print(type(test_labels[0]))
print(test_labels.shape)

<class 'numpy.ndarray'>
<class 'numpy.uint8'>
(50000, 1)
<class 'numpy.ndarray'>
(10000, 1)


In [114]:
train_labels = train_labels.reshape(-1)
test_labels = test_labels.reshape(-1)

In [115]:
print(train_labels.shape)
print(test_labels.shape)

(50000,)
(10000,)


In [116]:
train_labels = np.eye(10)[train_labels]
test_labels = np.eye(10)[test_labels]
print(train_labels.shape)
print(test_labels.shape)

(50000, 10)
(10000, 10)


In [100]:
# Split the data into training and validation sets
# this is already done

VGG16 (Visual Geometry Group 16) is a deep convolutional neural network architecture that was developed by the Visual Geometry Group at the University of Oxford. It was proposed by researchers Karen Simonyan and Andrew Zisserman in their paper titled "Very Deep Convolutional Networks for Large-Scale Image Recognition," which was presented at the International Conference on Learning Representations (ICLR) in 2015.

The VGG16 architecture gained significant popularity for its simplicity and effectiveness in image classification tasks. It was one of the pioneering models that demonstrated the power of deeper neural networks for visual recognition tasks.

Key characteristics of the VGG16 architecture:

1. Architecture: VGG16 consists of a total of 16 layers, hence the name "16." These layers are stacked one after another, forming a deep neural network.

2. Convolutional Layers: The main building blocks of VGG16 are the convolutional layers. It primarily uses 3x3 convolutional filters throughout the network, which allows it to capture local features effectively.

3. Max Pooling: After each set of convolutional layers, VGG16 applies max-pooling layers with 2x2 filters and stride 2, which halves the spatial dimensions (width and height) of the feature maps and reduces the number of parameters.

4. Fully Connected Layers: Towards the end of the network, VGG16 has fully connected layers that act as a classifier to make predictions based on the learned features.

5. Activation Function: The network uses the Rectified Linear Unit (ReLU) activation function for all hidden layers, which helps with faster convergence during training.

6. Number of Filters: The number of filters in each convolutional layer is relatively small compared to more recent architectures like ResNet or InceptionNet. However, stacking multiple layers allows VGG16 to learn complex hierarchical features.

7. Output Layer: The output layer consists of 1000 units, corresponding to 1000 ImageNet classes. VGG16 was originally trained on the large-scale ImageNet dataset, which contains millions of images from 1000 different classes.

VGG16 was instrumental in showing that increasing the depth of a neural network can significantly improve its performance on image recognition tasks. However, the main drawback of VGG16 is its high number of parameters, making it computationally expensive and memory-intensive to train. Despite this limitation, VGG16 remains an essential benchmark architecture and has paved the way for even deeper and more efficient models in the field of computer vision, such as ResNet, DenseNet, and EfficientNet.

Here are your tasks:

1. Load [VGG16](https://keras.io/api/applications/vgg/#vgg16-function) as a base model. Make sure to exclude the top layer.
2. Freeze all the layers in the base model. We'll be using these weights as a feature extraction layer to forward to layers that are trainable.

In [117]:
# Load the pre-trained VGG16 model (excluding the top classifier)
VGG_16_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [118]:
# Freeze the layers in the base model
for layer in VGG_16_model.layers:
    layer.trainable = False

Now, we'll add some trainable layers to the base model.

1. Using the base model, add a [GlobalAveragePooling2D](https://keras.io/api/layers/pooling_layers/global_average_pooling2d/) layer, followed by a [Dense](https://keras.io/api/layers/core_layers/dense/) layer of length 256 with ReLU activation. Finally, add a classification layer with 10 units, corresponding to the 10 CIFAR-10 classes, with softmax activation.
2. Create a Keras [Model](https://keras.io/api/models/model/) that takes in approproate inputs and outputs.

In [119]:
# Add a global average pooling layer
new_model = VGG_16_model.output
new_model = GlobalAveragePooling2D()(new_model)

In [120]:
# Add a fully connected layer with 256 units and ReLU activation
new_model = Dense(256, activation='relu')(new_model)

In [121]:
# Add the final classification layer with 10 units (for CIFAR-10 classes) and softmax activation
new_model = Dense(10, activation='softmax')(new_model)

In [122]:
# Create the fine-tuned model
model = Model(inputs=VGG_16_model.input, outputs=new_model)

With your model complete it's time to train it and assess its performance.

1. Compile your model using an appropriate loss function. Feel free to play around with the optimizer, but a good starting optimizer might be Adam with a learning rate of 0.001.
2. Fit your model on the training data. Use the validation data to print the accuracy for each epoch. Try training for 10 epochs. Note, training can take a few hours so go ahead and grab a cup of coffee.

**Optional**: See if you can implement an [Early Stopping](https://keras.io/api/callbacks/early_stopping/) criteria as a callback function.

In [123]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [124]:
# Train the model
model.fit(train_images, train_labels, epochs=10)

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 651s 415ms/step - accuracy: 0.4776 - loss: 1.4942
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 646s 413ms/step - accuracy: 0.5899 - loss: 1.1751
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 683s 413ms/step - accuracy: 0.6162 - loss: 1.1028
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 678s 411ms/step - accuracy: 0.6311 - loss: 1.0628
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 645s 413ms/step - accuracy: 0.6481 - loss: 1.0079
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 642s 411ms/step - accuracy: 0.6622 - loss: 0.9630
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 642s 411ms/step - accuracy: 0.6731 - loss: 0.9300
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 681s 410ms/step - accuracy: 0.6855 - loss: 0.8963
Epoch 9/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 685s 413ms/step - accuracy: 0.6987 - loss: 0.8602
Epoch 10/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 688s 416ms/step - accuracy: 0.7038 - loss: 0.8406


With your model trained, it's time to assess how well it performs on the test data.

1. Use your trained model to calculate the accuracy on the test set. Is the model performance better than random?
2. Experiment! See if you can tweak your model to improve performance.  

In [126]:
# Evaluate the model on the test set
test_accuracy = model.evaluate(test_images, test_labels)
print(test_accuracy)

313/313 ━━━━━━━━━━━━━━━━━━━━ 129s 414ms/step - accuracy: 0.6121 - loss: 1.1234
[1.1322460174560547, 0.6119999885559082]


In [ ]:
# The accuracy of 61% is better than a guess but I'm surprised at how bad it is.
# I want to revisit this and play around some
# I will try adjusting the learning rate, adding more layers, and training for longer
# Then I will try unfreezing some layers and fine tuning the best of those models
